In [1]:
!pip install textblob
!pip install tensorflow
!pip install Keras-Preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import re
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score, silhouette_score
from tensorflow.keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Dense, Activation, Dropout, BatchNormalization,Embedding

import warnings
warnings.filterwarnings("ignore")

In [3]:
test_preprocessed_file_path = "../input/dataset-preprocessed/test_data_preprocessed_11pm_12_12.csv"
train_preprocessed_file_path = "../input/dataset-preprocessed/train_data_preprocessed_1130pm_12_12.csv"
articles_testing_file_path = "../input/dataset-preprocessed/articles_testing.tsv"

In [4]:
df_preprocessed_data = pd.read_csv(train_preprocessed_file_path)
print(df_preprocessed_data.shape)
print(df_preprocessed_data.describe())
df_preprocessed_data.head()

(97846, 5)
          tag_count
count  97846.000000
mean      19.969033
std        0.507697
min        4.000000
25%       20.000000
50%       20.000000
75%       20.000000
max       23.000000


,content,tags,preprocessed_content,preprocessed_tags,tag_count
0,"Bước vào trận đấu, Barcelona nhanh chóng tràn ...","La Liga,Sevilla,Olimpic Lluis Companys,đè bẹp,...",trận đấu barcelona nhanh_chóng tràn tấn_công v...,"la_liga,sevilla,olimpic_lluis_companys,đè_bẹp,...",20
1,Willian đi vào lịch sử bóng đá xứ samba. Với 1...,"Estevao Willian,Neymar,giải VĐQG Brazil,Serie ...",willian đi lịch_sử bóng_đá xứ samba 1 bàn thắn...,"estevao_willian,neymar,giải_vđqg_brazil,serie_...",20
2,Giải vô địch ná cao su thế giới năm 2024 đã di...,"ná cao su,giải vô địch,giải Ba,Thượng Hải,đồng...",giải vô_địch ná cao_su thế_giới năm 2024 diễn ...,"ná_cao_su,giải_vô_địch,giải_ba,thượng_hải,đồng...",19
3,Mục tiêu lớn Phó Thủ tướng Chính phủ Lê Thành ...,"đăng cai,Đại hội Thể thao châu Á,ASIAD,thể dục...",mục_tiêu phó thủ_tướng chính_phủ lê_thành_long...,"đăng_cai,đại_hội_thể_thao_châu_á,asiad,thể_dục...",20
4,"Ngày 20-10, Giải bơi và lặn vô địch quốc gia n...","toàn đoàn,lặn,Vũ Đặng Nhật Nam,Nguyễn Lê Truyề...",20 10 giải bơi lặn vô_địch quốc_gia năm 2024 k...,"toàn_đoàn,lặn,vũ_đặng_nhật_nam,nguyễn_lê_truyề...",20


In [5]:
preprocessed_df = df_preprocessed_data[['preprocessed_content', 'preprocessed_tags']]
print(preprocessed_df.shape)
preprocessed_df.head()

(97846, 2)


,preprocessed_content,preprocessed_tags
0,trận đấu barcelona nhanh_chóng tràn tấn_công v...,"la_liga,sevilla,olimpic_lluis_companys,đè_bẹp,..."
1,willian đi lịch_sử bóng_đá xứ samba 1 bàn thắn...,"estevao_willian,neymar,giải_vđqg_brazil,serie_..."
2,giải vô_địch ná cao_su thế_giới năm 2024 diễn ...,"ná_cao_su,giải_vô_địch,giải_ba,thượng_hải,đồng..."
3,mục_tiêu phó thủ_tướng chính_phủ lê_thành_long...,"đăng_cai,đại_hội_thể_thao_châu_á,asiad,thể_dục..."
4,20 10 giải bơi lặn vô_địch quốc_gia năm 2024 k...,"toàn_đoàn,lặn,vũ_đặng_nhật_nam,nguyễn_lê_truyề..."


In [6]:
def count_words(text):
    return len(text.split()) if isinstance(text, str) else 0

In [7]:
max_content_len = preprocessed_df['preprocessed_content'].apply(count_words).max()
print(max_content_len)

8185


In [8]:
vectorizer = CountVectorizer(max_features=100000)
tfidf_matrix = vectorizer.fit_transform(preprocessed_df['preprocessed_content'])

In [9]:
optimal_clusters = 6
# Thực hiện phân cụm K-Means với số cụm tối ưu
kmeans_optimal = KMeans(n_clusters=optimal_clusters, random_state=42, n_init=10)
preprocessed_df['cluster'] = kmeans_optimal.fit_predict(tfidf_matrix)

cluster_distribution = preprocessed_df['cluster'].value_counts()

optimal_clusters, cluster_distribution

(6,
 cluster
 0    76519
 5    10231
 1     3178
 4     2901
 2     2550
 3     2467
 Name: count, dtype: int64)

In [10]:

total_samples = int(0.75 * len(preprocessed_df))

samples_per_cluster = (cluster_distribution / cluster_distribution.sum() * total_samples).round().astype(int)


sampled_data = pd.DataFrame()

for cluster_id, n_samples in samples_per_cluster.items():
    cluster_samples = preprocessed_df[preprocessed_df['cluster'] == cluster_id].sample(n=n_samples, random_state=42)
    sampled_data = pd.concat([sampled_data, cluster_samples])

sampled_data = sampled_data.reset_index(drop=True)
sampled_data.shape, sampled_data['cluster'].value_counts()

((73383, 3),
 cluster
 0    57389
 5     7673
 1     2383
 4     2176
 2     1912
 3     1850
 Name: count, dtype: int64)

In [11]:
sampled_data.head()

,preprocessed_content,preprocessed_tags,cluster
0,họp_báo 15 10 phóng_viên afp đề_nghị trung_quố...,"mao_ninh,triều_tiên-hàn_quốc,bán_đảo_triều_tiê...",0
1,dự lễ uỷ_viên ban_chấp_hành trung_ương đảng bí...,"văn_phòng_quốc_hội,biểu_nhân_dân,vũ_thị_lan_an...",0
2,sức gió tâm_bão 11 giật 14 bão di_chuyển hướng...,"áp_sát,tâm_bão,bão,độ_kinh_đông,vào_hồi,độ_vĩ_...",0
3,đặc_trưng thế_hệ thiếu_nhi thành_phố thời 4 0 ...,"truyện_thiếu_nhi,đại_náo,ông_ngoại,ngúng_nguẩy...",0
4,thuý_diễm tác_phẩm điện_ảnh trình_làng bà_già ...,"thúy_diễm,út_thà,mưu_cầu,trước_nay,địa_hạt,nữ_...",0


In [12]:
df_train = sampled_data
print(df_train.shape)
df_train.head()

(73383, 3)


,preprocessed_content,preprocessed_tags,cluster
0,họp_báo 15 10 phóng_viên afp đề_nghị trung_quố...,"mao_ninh,triều_tiên-hàn_quốc,bán_đảo_triều_tiê...",0
1,dự lễ uỷ_viên ban_chấp_hành trung_ương đảng bí...,"văn_phòng_quốc_hội,biểu_nhân_dân,vũ_thị_lan_an...",0
2,sức gió tâm_bão 11 giật 14 bão di_chuyển hướng...,"áp_sát,tâm_bão,bão,độ_kinh_đông,vào_hồi,độ_vĩ_...",0
3,đặc_trưng thế_hệ thiếu_nhi thành_phố thời 4 0 ...,"truyện_thiếu_nhi,đại_náo,ông_ngoại,ngúng_nguẩy...",0
4,thuý_diễm tác_phẩm điện_ảnh trình_làng bà_già ...,"thúy_diễm,út_thà,mưu_cầu,trước_nay,địa_hạt,nữ_...",0


In [13]:
df_train['preprocessed_tags'] = df_train['preprocessed_tags'].str.split(',')

In [14]:
df_train.head()

,preprocessed_content,preprocessed_tags,cluster
0,họp_báo 15 10 phóng_viên afp đề_nghị trung_quố...,"[mao_ninh, triều_tiên-hàn_quốc, bán_đảo_triều_...",0
1,dự lễ uỷ_viên ban_chấp_hành trung_ương đảng bí...,"[văn_phòng_quốc_hội, biểu_nhân_dân, vũ_thị_lan...",0
2,sức gió tâm_bão 11 giật 14 bão di_chuyển hướng...,"[áp_sát, tâm_bão, bão, độ_kinh_đông, vào_hồi, ...",0
3,đặc_trưng thế_hệ thiếu_nhi thành_phố thời 4 0 ...,"[truyện_thiếu_nhi, đại_náo, ông_ngoại, ngúng_n...",0
4,thuý_diễm tác_phẩm điện_ảnh trình_làng bà_già ...,"[thúy_diễm, út_thà, mưu_cầu, trước_nay, địa_hạ...",0


In [15]:
all_tags = [tag for tags in df_train['preprocessed_tags'] for tag in tags]
common_tags = [tag for tag, count in Counter(all_tags).most_common(10214)] 

In [16]:
df_train['filtered_tags'] = df_train['preprocessed_tags'].apply(lambda tags: [tag for tag in tags if tag in common_tags])

In [17]:
print(df_train.shape)
df_train.head()

(73383, 4)


,preprocessed_content,preprocessed_tags,cluster,filtered_tags
0,họp_báo 15 10 phóng_viên afp đề_nghị trung_quố...,"[mao_ninh, triều_tiên-hàn_quốc, bán_đảo_triều_...",0,"[bán_đảo_triều_tiên, đường_sắt_liên_triều, ngư..."
1,dự lễ uỷ_viên ban_chấp_hành trung_ương đảng bí...,"[văn_phòng_quốc_hội, biểu_nhân_dân, vũ_thị_lan...",0,"[văn_phòng_quốc_hội, biểu_nhân_dân, báo_đại, b..."
2,sức gió tâm_bão 11 giật 14 bão di_chuyển hướng...,"[áp_sát, tâm_bão, bão, độ_kinh_đông, vào_hồi, ...",0,"[áp_sát, tâm_bão, bão, độ_kinh_đông, vào_hồi, ..."
3,đặc_trưng thế_hệ thiếu_nhi thành_phố thời 4 0 ...,"[truyện_thiếu_nhi, đại_náo, ông_ngoại, ngúng_n...",0,"[nhà_xuất_bản_kim_đồng, người_lớn, kỳ_nghỉ, tá..."
4,thuý_diễm tác_phẩm điện_ảnh trình_làng bà_già ...,"[thúy_diễm, út_thà, mưu_cầu, trước_nay, địa_hạ...",0,"[thúy_diễm, nữ_hoàng, phim_truyền_hình, điện_ả..."


In [18]:
tags = df_train['filtered_tags'].values
tags = [tag for tag in tags]

In [19]:
mlb = MultiLabelBinarizer()
mlb.fit(df_train['filtered_tags'])
mlb


MultiLabelBinarizer()

In [20]:
tags_list = mlb.classes_
tags_list

array(["'em", '24k', '2g', ..., 'ứng_tuyển', 'ứng_viên', 'ứng_xử'],
      dtype=object)

In [21]:
y = mlb.transform(df_train['filtered_tags'])
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [22]:
X = df_train['preprocessed_content']
X

0        họp_báo 15 10 phóng_viên afp đề_nghị trung_quố...
1        dự lễ uỷ_viên ban_chấp_hành trung_ương đảng bí...
2        sức gió tâm_bão 11 giật 14 bão di_chuyển hướng...
3        đặc_trưng thế_hệ thiếu_nhi thành_phố thời 4 0 ...
4        thuý_diễm tác_phẩm điện_ảnh trình_làng bà_già ...
                               ...                        
73378    chiều 30 7 trụ_sở văn_phòng chính_phủ phó thủ_...
73379    rào_cản truyện_tranh lĩnh_vực công_nghiệp hình...
73380    pgs ts vũ_văn_phúc nhà_nước pháp_quyền xhcn mô...
73381    thay_mặt lãnh_đạo công_an bộ_trưởng lương_tam_...
73382    21 10 phiên khai_mạc kỳ họp 8 quốc_hội khoá xv...
Name: preprocessed_content, Length: 73383, dtype: object

In [23]:
# train test split protect overfitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
len(X_train), len(X_test), len(y_train), len(y_test)

(66044, 7339, 66044, 7339)

In [24]:
def count_words(text):
    return len(text.split()) if isinstance(text, str) else 0

In [25]:
max_content_sample_len = df_train['preprocessed_content'].apply(count_words).max()
print(max_content_sample_len)

8185


In [26]:
X_train.shape,y_train.shape

((66044,), (66044, 10214))

In [27]:
tokenizer = Tokenizer(lower=True)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = pad_sequences(X_train,padding= 'post',maxlen = max_content_sample_len)
X_test = pad_sequences(X_test,padding='post', maxlen = max_content_sample_len)

In [28]:
# cnn model
model_CNN = Sequential()

model_CNN.add(Embedding(len(tokenizer.word_index) + 1,50,input_length=max_content_sample_len))
model_CNN.add(Dropout(0.2))
model_CNN.add(Conv1D(256,3,padding='valid',activation='relu', strides=1))
model_CNN.add(GlobalMaxPooling1D())
model_CNN.add(Dense(256))
model_CNN.add(Dropout(0.2))
model_CNN.add(Activation('relu'))
model_CNN.add(Dense(y_train.shape[1]))
model_CNN.add(Activation('sigmoid'))

model_CNN.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model_CNN.fit(X_train, y_train,batch_size=32,epochs=30,validation_data=(X_test, y_test))


temp_array = np.arange(0.05,0.5,0.05)
scores = []

for probability in temp_array:
    predict_cnn = model_CNN.predict(X_test)
    predict_cnn[predict_cnn>=probability] = 1
    predict_cnn[predict_cnn<probability] = 0
    f1_samples = f1_score(y_test, predict_cnn, average="samples")
    f1_micro = f1_score(y_test, predict_cnn, average="micro")
    f1_macro= f1_score(y_test, predict_cnn, average="macro")
    
    scores.append((f1_samples, f1_micro, f1_macro, probability))
    print("=============================================================")
    print("Accuracy :",metrics.accuracy_score(y_test, predict_cnn))
    print(f"F1_samples: {f1_samples}, F1_micro: {f1_micro}, F1_macro: {f1_macro}, Threshold: {probability}")
    print("Hamming loss :", metrics.hamming_loss(y_test, predict_cnn))
    print("=============================================================")

Epoch 1/30


I0000 00:00:1734096132.637210     116 service.cc:145] XLA service 0x7c777813b630 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734096132.637300     116 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1734096132.637306     116 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


   2/2064 ━━━━━━━━━━━━━━━━━━━━ 2:50 83ms/step - accuracy: 0.0000e+00 - loss: 0.6928 

I0000 00:00:1734096138.465033     116 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2064/2064 ━━━━━━━━━━━━━━━━━━━━ 114s 51ms/step - accuracy: 4.5294e-04 - loss: 0.0338 - val_accuracy: 0.0000e+00 - val_loss: 0.0279
Epoch 2/30
2064/2064 ━━━━━━━━━━━━━━━━━━━━ 106s 51ms/step - accuracy: 0.0011 - loss: 0.0088 - val_accuracy: 0.0069 - val_loss: 0.0162
Epoch 3/30
2064/2064 ━━━━━━━━━━━━━━━━━━━━ 106s 52ms/step - accuracy: 0.0094 - loss: 0.0076 - val_accuracy: 0.0275 - val_loss: 0.0084
Epoch 4/30
2064/2064 ━━━━━━━━━━━━━━━━━━━━ 106s 52ms/step - accuracy: 0.0259 - loss: 0.0065 - val_accuracy: 0.0350 - val_loss: 0.0067
Epoch 5/30
2064/2064 ━━━━━━━━━━━━━━━━━━━━ 106s 52ms/step - accuracy: 0.0330 - loss: 0.0060 - val_accuracy: 0.0409 - val_loss: 0.0060
Epoch 6/30
2064/2064 ━━━━━━━━━━━━━━━━━━━━ 106s 52ms/step - accuracy: 0.0363 - loss: 0.0056 - val_accuracy: 0.0458 - val_loss: 0.0055
Epoch 7/30
2064/2064 ━━━━━━━━━━━━━━━━━━━━ 106s 52ms/step - accuracy: 0.0427 - loss: 0.0054 - val_accuracy: 0.0391 - val_loss: 0.0054
Epoch 8/30
2064/2064 ━━━━━━━━━━━━━━━━━━━━ 106s 52ms/step - accuracy: 0.0

In [29]:
df_test = pd.read_csv(test_preprocessed_file_path)

In [30]:
max_content_test_sample_len = df_test['preprocessed_content'].apply(count_words).max()
print(max_content_test_sample_len)

1047


In [31]:
tags_predict_list = []
num = 0
for content in df_test['preprocessed_content']:
    if num%10 == 0:
        print(num)
    num+=1
    sequence = tokenizer.texts_to_sequences([content])
    padded_sequence = pad_sequences(sequence, padding='post', maxlen=max_content_test_sample_len)
    prediction = model_CNN.predict(padded_sequence)
    prediction = prediction.flatten()
    prediction_threshold = 0.15
    predicted_tags = mlb.classes_[prediction >= prediction_threshold]
    tags_predict = ",".join(tag.replace("_", " ") for tag in predicted_tags)
    tags_predict_list.append(tags_predict)

0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
27900
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
27910
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
27920
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
32320
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
32330
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
36760
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
36770
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
36780
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms

In [32]:
df_result = pd.read_csv(articles_testing_file_path, sep='\t')
df_result['tags'] = tags_predict_list

In [33]:
df_result.head()

,content,tags
0,"Vào tối 22/10, sau đêm bán kết của cuộc thi Ho...","dân tộc,hoa hậu,hoa hậu quế anh,khảm xà cừ,mis..."
1,"Tân Hoa Xã đưa tin rằng vào ngày 9/10, phát ng...","bộ ngoại giao nga,cáo buộc,liên bang nga,liên ..."
2,Chuyến thăm của Lloyd Austin diễn ra chỉ vài n...,"bầu cử,bộ trưởng quốc phòng,harris,joe biden,k..."
3,"Trong dự thảo Luật Nhà giáo, Bộ GD&ĐT đã đưa r...","bộ gd,công khai,dự thảo,dự tuyển,giáo viên,luậ..."
4,"Trong buổi gặp gỡ, Chủ tịch Hà Thị Nga đã bày ...","cuba,hội lhpn việt nam,ngày phụ nữ việt nam,nă..."


In [34]:
df_result.to_csv('cnn_result.tsv', sep='\t', index=False)